# Daily Stock Screener

Notes:
- NASDAQ trading time is 9:30 am to 4 pm (GMT-4)

## Imports

In [1]:
# general module import
import pandas as pd
import numpy as np

In [2]:
# Local spark cluster specific imports for Windows
import findspark
findspark.init()

In [3]:
# spark specific import. 
# Note: wait until run of previous cell is complete to avoid start-up issues.
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()
df = spark.sql("select 'spark' as hello ")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [4]:
# import of own packages
from analysis_utils import read_result_tables
from get_prices import *

## Extract Source Data

In [5]:
# extract screener results
company_info_df, fundamental_df, growth_df, screener_df = read_result_tables()

In [6]:
# get symbol list from growth_df
symbol_list = growth_df['Ticker'].unique().tolist()

In [10]:
# get daily stock data
daily_prices_path = '../data/6_daily_prices'
download_ticker_prices(symbol_list, period='1d', download_path=daily_prices_path)

Start download of historic price data
Ticker price data extracted...
Total number of valid symbols downloaded = 1523


In [8]:
# load daily stock prides from disk
price_df = load_ticker_prices(spark, symbol_list, source_path=daily_prices_path)

## Transform to daily screener data

In [9]:
# update screener df with daily price data
screener_df = find_stocks_below_mos(spark, price_df, growth_df)

## Load Screener results to disk

In [ ]:
screener_df.to_parquet('..//data//5_results//' + 'screener' + '.parquet.gzip', compression='gzip')